<a href="https://colab.research.google.com/github/victorteodoro/projeto-aplicado-ii/blob/main/5_Treinando_e_testando_o_modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Abrindo o projeto

Abrimos o projeto para ler lista de categorias e seus índices definidos pelo conjunto de treinamento.

In [6]:
!wget https://asav.com.br/carcara/modulo.py

with open("modulo.py", 'r') as file:
    exec(file.read())

import getpass

CKEY = getpass.getpass('Digite a chave de acesso: ')

print("\n-----------------------\n")

projeto = ProjetoCarcara("2024-1-agua-e-vida-stl", ftp=ftplib.FTP("ftp.asav.com.br", "datascience@asav.com.br", CKEY))

abrir_mapa = 2  # escolhemos o mapa de índice 2

print(projeto.galeria['data'].iloc[abrir_mapa]+"\n")

imagem = projeto.abrirMapa(projeto.galeria['mapa_id'].iloc[abrir_mapa]) # imagem de satélite baixada
img=imagem.visualizarMapa(projeto.area_interesse[0], projeto.area_interesse[1], projeto.area_interesse[2], projeto.area_interesse[3])

mascara = imagem.mascaras['objeto'].iloc[0]

display(mascara.categorias)

--2024-03-29 21:06:59--  https://asav.com.br/carcara/modulo.py
Resolving asav.com.br (asav.com.br)... 108.179.252.164
Connecting to asav.com.br (asav.com.br)|108.179.252.164|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45136 (44K)
Saving to: ‘modulo.py.2’

modulo.py.2         100%[===================>]  44.08K  --.-KB/s    in 0.04s   

2024-03-29 21:06:59 (1.13 MB/s) - ‘modulo.py.2’ saved [45136/45136]

Digite a chave de acesso: ··········


# Abrindo o conjunto de treinamento

Insira abaixo o arquivo de treinamento baixado na etapa anterior:

In [3]:
upload = files.upload()

nome_do_arquivo = next(iter(upload))
arquivo = io.BytesIO(upload[nome_do_arquivo]).getvalue()
print(nome_do_arquivo)

Saving S0000025fO0000048b.gz to S0000025fO0000048b.gz


**OU** abra ele do colab diretamente se já tiver feito o *upload*.

In [13]:
import io

# Caminho para o arquivo que você quer abrir
caminho_do_arquivo = '/content/S0000025fO0000048b.gz' # Altere se necessário

# Lendo o arquivo e armazenando seu conteúdo em um objeto BytesIO
with open(caminho_do_arquivo, 'rb') as arquivo:
    conteudo = arquivo.read()
    arquivo = io.BytesIO(conteudo).getvalue()

# Preparando o conjunto de treinamento

Agora vamos formatar os dados para o conjunto de treinamento dentro da biblioteca *numpy*.

In [16]:
import numpy as np
import struct

# Supondo que `arquivo` é seu array de bytes e `mascara.categorias['indice']` são as categorias desejadas

# Processar entradas
numero_de_entradas = len(arquivo) // tam_item
entradas_np = np.zeros((numero_de_entradas, 12, 12, 3), dtype=np.float32)

for i in range(numero_de_entradas):
    inicio = i * tam_item
    entrada_bytes = arquivo[inicio:inicio+tam_entrada]
    entrada_array = np.frombuffer(entrada_bytes, dtype=np.uint8).reshape(12, 12, 3) / 255.0
    entradas_np[i] = entrada_array

# Processar saídas
saidas_np = np.zeros((numero_de_entradas, 4, 4, 1), dtype=np.float32)

for i, saida_pre in enumerate(saidas_pre):
    # Convertendo índices de categorias em formato one-hot 4x4x1
    saida_array = np.zeros((4, 4, 1), dtype=np.float32)
    for y in range(4):
        for x in range(4):
            indice = y * 4 + x
            categoria = saida_pre[indice]
            if categoria in categorias:  # Supondo que `categorias` é uma lista de índices válidos
                saida_array[y, x, 0] = 1.0
    saidas_np[i] = saida_array

# Agora `entradas_np` e `saidas_np` estão prontos para serem utilizados


Agora temos o conjunto de entrada e o de saída já formatados em duas listas de listas.

**Para a entrada** os valores R, G e B em uma matriz 12x12.

**Para a saída** a probabilidade de cada categoria em uma matriz 4x4.

# Treinando e avaliando o modelo

Agora vamos ao *tensorflow* e ao *keras* para treinar o modelo.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Reshape
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
from google.colab import files
from datetime import datetime

# Dividindo os dados em conjuntos de treinamento e teste
entradas_treinamento, entradas_teste, saidas_treinamento, saidas_teste = train_test_split(
    entradas_np, saidas_np, test_size=0.5, random_state=42)

# Número de categorias
num_classes = 16  # Ajuste conforme o número exato de categorias

# Definindo o modelo
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(12, 12, 3)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    # Ajuste para problemas de múltiplas etiquetas binárias com uma camada Reshape
    Dense(4*4*1, activation='sigmoid'),
    Reshape((4, 4, 1))  # Correção aqui
])

# Compilar o modelo
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Treinar o modelo
model.fit(entradas_treinamento, saidas_treinamento, validation_data=(entradas_teste, saidas_teste), epochs=10, batch_size=32)

# Avaliar o modelo no conjunto de teste
avaliacao_teste = model.evaluate(entradas_teste, saidas_teste)
print(f"Test Loss: {avaliacao_teste[0]}, Test Accuracy: {avaliacao_teste[1]}")


Epoch 1/10
18818/18818 [==============================] - 136s 7ms/step - loss: 0.3083 - accuracy: 0.8507 - val_loss: 0.2677 - val_accuracy: 0.8793
Epoch 2/10
18818/18818 [==============================] - 127s 7ms/step - loss: 0.2549 - accuracy: 0.8860 - val_loss: 0.2429 - val_accuracy: 0.8919
Epoch 3/10
18818/18818 [==============================] - 127s 7ms/step - loss: 0.2409 - accuracy: 0.8924 - val_loss: 0.2361 - val_accuracy: 0.8947
Epoch 4/10
18818/18818 [==============================] - 131s 7ms/step - loss: 0.2326 - accuracy: 0.8966 - val_loss: 0.2285 - val_accuracy: 0.8980
Epoch 5/10
18818/18818 [==============================] - 135s 7ms/step - loss: 0.2263 - accuracy: 0.8998 - val_loss: 0.2165 - val_accuracy: 0.9041
Epoch 6/10
18818/18818 [==============================] - 130s 7ms/step - loss: 0.2210 - accuracy: 0.9023 - val_loss: 0.2159 - val_accuracy: 0.9053
Epoch 7/10
18818/18818 [==============================] - 137s 7ms/step - loss: 0.2158 - accuracy: 0.9049 - val_

Podemos repetir o treinamento se achar necessário

In [ ]:
# Treinar o modelo
model.fit(entradas_treinamento, saidas_treinamento, validation_data=(entradas_teste, saidas_teste), epochs=10, batch_size=32)

# Avaliar o modelo no conjunto de teste
avaliacao_teste = model.evaluate(entradas_teste, saidas_teste)
print(f"Test Loss: {avaliacao_teste[0]}, Test Accuracy: {avaliacao_teste[1]}")

Por fim, salvar o modelo treinado

In [18]:
# Salvar o modelo treinado
nome_do_arquivo = f"seriema-{datetime.now().strftime('%Y-%m-%d_%H-%M-%S-%f')}.h5"
model.save(nome_do_arquivo)

# Usar a função files.download para oferecer o arquivo para download
files.download(nome_do_arquivo)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>